# Funciones 

In [248]:
import pandas as pd

df_reviews = pd.read_parquet("reviews_eda.parquet")
df_items = pd.read_parquet("items_eda.parquet")
df_games = pd.read_parquet("games_eda.parquet")

# PlayTimeGenre
    def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [249]:
# Modificaciones a los dataframes para mejorar el rendimiento de las funciones
playtiem_item = df_items.groupby('item_id')['playtime_forever'].sum().reset_index()
df_merge = pd.merge(df_games, playtiem_item, on='item_id', how='inner')
df_PlayTimeGenre = df_merge[["genres","year","playtime_forever"]]
df_PlayTimeGenre =df_PlayTimeGenre.dropna()
df_PlayTimeGenre.to_parquet(r'df_PlayTimeGenre.parquet',index=False)

In [250]:

def PlayTimeGenre(genero : str):
    # Filtrar el DataFrame para obtener solo las filas que contienen el género específico
    filtro_genero = df_PlayTimeGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_PlayTimeGenre[filtro_genero]
    df_agrupado = df_filtrado.groupby('year')['playtime_forever'].sum().reset_index()
    # Encontrar el año con la mayor suma de 'playtime_forever'
    df_agrupado['year'] = df_agrupado['year'].astype(int)
    anio_mayor_suma = df_agrupado.loc[df_agrupado['playtime_forever'].idxmax(), 'year']
    respuesta = {f"Año de lanzamiento con más horas jugadas para Género el {genero}": anio_mayor_suma}
    return respuesta 

PlayTimeGenre("Indie")

{'Año de lanzamiento con más horas jugadas para Género el Indie': 2011}

# UserForGenre


    def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [263]:
df_UserForGenre

,user_id,genres,year,playtime_forever
0,76561197970982479,['Action'],2000.0,6
1,doctr,['Action'],2000.0,93
2,corrupted_soul,['Action'],2000.0,108
3,WeiEDKrSat,['Action'],2000.0,328
5,Fr0stedLine,['Action'],2000.0,580
...,...,...,...,...
2644071,76561198107283457,['Simulation'],2016.0,14
2644072,76561198107283457,"['Action', 'Casual', 'Indie', 'Simulation']",2016.0,164
2644073,Miami_Max,['Utilities'],2016.0,1
2644074,valinorau,"['Action', 'Indie', 'RPG', 'Simulation', 'Stra...",2015.0,20


In [262]:
df_aux = df_games[["item_id","genres","year"]]
df_UserForGenre = pd.merge(df_items,df_aux, on = "item_id")
df_UserForGenre =df_UserForGenre[["user_id","genres","year","playtime_forever"]]
def eliminar_outliers_iqr(df, columna):
    # Calcular el rango intercuartílico (IQR)
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites superior e inferior para identificar outliers
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Filtrar el DataFrame para mantener solo los valores dentro de los límites
    df_filtrado = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]

    return df_filtrado

# Ejemplo de uso
# Supongamos que 'df' es tu DataFrame y 'valor' es la columna que contiene los datos
df_UserForGenre = eliminar_outliers_iqr(df_UserForGenre, 'playtime_forever')
df_UserForGenre.to_parquet(r'df_UserForGenre.parquet',index=False)

In [252]:
def userforgenre(genero: str):
    # Filtrar el DataFrame para el género especificado
    filtro_genero = df_UserForGenre['genres'].str.contains(genero, case=False, na=False)
    df_filtrado = df_UserForGenre[filtro_genero]

    # Agrupar por 'user_id' y 'year', sumar las horas jugadas
    df_agrupado = df_filtrado.groupby(['user_id', 'year'])['playtime_forever'].sum().reset_index()

    # Encontrar el usuario con la máxima suma de horas jugadas
    idx_max_playtime = df_agrupado['playtime_forever'].idxmax()
    usuario_max_playtime = df_agrupado.loc[idx_max_playtime, 'user_id']

    # Filtrar el DataFrame para el usuario con máxima suma de horas jugadas
    df_usuario = df_agrupado[df_agrupado['user_id'] == usuario_max_playtime]

    # Crear el formato "Horas jugadas"
    resultado_final = [{'Año': int(row['year']), 'Horas': int(row['playtime_forever'])} for _, row in df_usuario.iterrows()]
    
    return {"Usuario con más horas jugadas para Género {}:".format(genero): usuario_max_playtime, "Horas jugadas": resultado_final}

# Ejemplo de uso
genero_a_buscar = 'action'
resultado = userforgenre(genero_a_buscar)
print(resultado)


{'Usuario con más horas jugadas para Género action:': 'REBAS_AS_F-T', 'Horas jugadas': [{'Año': 1998, 'Horas': 2}, {'Año': 1999, 'Horas': 225}, {'Año': 2001, 'Horas': 11}, {'Año': 2002, 'Horas': 1}, {'Año': 2003, 'Horas': 1863}, {'Año': 2004, 'Horas': 2115}, {'Año': 2005, 'Horas': 3293}, {'Año': 2006, 'Horas': 138}, {'Año': 2007, 'Horas': 684}, {'Año': 2008, 'Horas': 3791}, {'Año': 2009, 'Horas': 26635}, {'Año': 2010, 'Horas': 31315}, {'Año': 2011, 'Horas': 72378}, {'Año': 2012, 'Horas': 115563}, {'Año': 2013, 'Horas': 180725}, {'Año': 2014, 'Horas': 267280}, {'Año': 2015, 'Horas': 218917}, {'Año': 2016, 'Horas': 263522}, {'Año': 2017, 'Horas': 3716}]}


# UsersRecommend


    def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [253]:
df_reviews

,user_id,user_url,item_id,helpful,recommend,review,posted_date,sentimiento,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,0.8481,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,No ratings yet,True,It's unique and worth a playthrough.,2011-07-15,0.2263,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011-04-21,0.9117,2
3,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,0.9566,2
4,js41637,http://steamcommunity.com/id/js41637,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013-09-08,0.9708,2
...,...,...,...,...,...,...,...,...,...
48466,wayfeng,http://steamcommunity.com/id/wayfeng,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN,2015-10-14,0.0000,1
48467,76561198251004808,http://steamcommunity.com/profiles/76561198251...,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...,2015-10-10,0.7650,2
48468,72947282842,http://steamcommunity.com/id/72947282842,730,No ratings yet,True,Prettyy Mad Game,2015-10-31,-0.4939,0
48469,ApxLGhost,http://steamcommunity.com/id/ApxLGhost,730,No ratings yet,True,AMAZING GAME 10/10,2015-12-14,0.6739,2


In [254]:
df_reviews["posted_date"] = pd.to_datetime(df_reviews["posted_date"])
df_reviews["year_posted"] = df_reviews["posted_date"].dt.year
df_UserRecommend = df_reviews[['item_id',"recommend",'year_posted']]
df_aux= df_games[["item_id","title"]]
df_UserRecommend = pd.merge(df_UserRecommend,df_aux, on="item_id", how="inner")
df_UserRecommend = df_UserRecommend[['title','year_posted',"recommend"]]
df_UserRecommend.to_parquet(r'df_UserRecommend.parquet',index=False)

In [255]:
def UsersRecommend(año: int):
    
    df = df_UserRecommend[(df_UserRecommend['year_posted'] == año) & (df_UserRecommend['recommend'] == True)]

    # Contar el número de recomendaciones para cada juego
    top_juegos = df['title'].value_counts().head(3)

    # Crear el formato de salida
    resultado_final = [{"Puesto {}: {}".format(i+1, juego): recomendaciones} for i, (juego, recomendaciones) in enumerate(top_juegos.items())]

    return resultado_final

# Ejemplo de uso
año_a_consultar = 2014  # Puedes ajustar este valor según tu necesidad
resultado = UsersRecommend(año_a_consultar)
print(resultado)

[{'Puesto 1: Team Fortress 2': 1546}, {'Puesto 2: Counter-Strike: Global Offensive': 1067}, {"Puesto 3: Garry's Mod": 757}]


# def UsersWorstDeveloper


    def UsersWorstDeveloper( año : int ): Devuelve el top 3 de desarrolladoras con juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [284]:
df_reviews.head(5)
df_UserWorstDeveloper = df_reviews[["item_id","recommend","year_posted"]]
df_info = df_games[["item_id","developer"]]
df_UserWorstDeveloper = pd.merge(df_UserWorstDeveloper,df_info, on ="item_id",how= "inner")
df_UserWorstDeveloper = df_UserWorstDeveloper[(df_UserWorstDeveloper['recommend'] == False)]
df_UserWorstDeveloper = df_UserWorstDeveloper[["developer", "year_posted"]]
df_UserWorstDeveloper.to_parquet(r'df_UserWorstDeveloper.parquet',index=False)
df_UserWorstDeveloper.iloc[0,0]

'Tripwire Interactive'

In [310]:

def UserWorstDeveloper(year):
    # Filtrar el DataFrame por el año especificado
    df_filtered = df_UserWorstDeveloper[df_UserWorstDeveloper['year_posted'] == year]

    # Contar la cantidad de publicaciones por desarrollador
    developer_counts = df_filtered['developer'].value_counts()

    # Tomar los primeros N resultados
    top_developers = developer_counts.head(3)

    # Crear la lista con el formato solicitado
    result_list = [{"Puesto {}: {}".format(i + 1, developer): count} for i, (developer, count) in enumerate(top_developers.items())]

    return print(result_list)

# Ejemplo de uso
year_to_filter = 2011
UserWorstDeveloper(year_to_filter)


[{'Puesto 1: Valve': 3}, {'Puesto 2: Broken Rules': 2}, {'Puesto 3: Ubisoft Montpellier': 1}]


# Sentiment_analysis


    def sentiment_analysis( empresa desarrolladora : str ): Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.

Ejemplo de retorno: {'Valve' : [Negative = 182, Neutral = 120, Positive = 278]}

In [258]:
df_10 = df_reviews[["item_id","sentiment_analysis"]]
df_11 = df_games[["developer","item_id"]]
df_Sentiment_Analysis = pd.merge(df_10,df_11, on = "item_id", how= "inner") 
df_Sentiment_Analysis = df_Sentiment_Analysis.groupby(['developer', 'sentiment_analysis']).size().unstack(fill_value=0)
df_Sentiment_Analysis = df_Sentiment_Analysis.reset_index(drop=False)
nuevos_nombres = ["Developer",'Negativo', 'Neutro',"Positivo"]
df_Sentiment_Analysis.columns = nuevos_nombres
df_Sentiment_Analysis["Developer"] = df_Sentiment_Analysis["Developer"].astype(str).str.lower()
df_Sentiment_Analysis.to_parquet("df_Sentiment_Analysis.parquet", index=False )

In [259]:
def Sentiment_analysis(desarrollador):
    # Filtra el DataFrame para obtener la fila correspondiente al desarrollador
    desarrollador =desarrollador.lower()
    desarrollador_fila = df_Sentiment_Analysis[df_Sentiment_Analysis['Developer'] == desarrollador]

    # Extrae los valores de las columnas Negativo, Neutro y Positivo
    negativo = desarrollador_fila['Negativo'].values[0]
    neutro = desarrollador_fila['Neutro'].values[0]
    positivo = desarrollador_fila['Positivo'].values[0]

    # Crea un diccionario con la información en el formato solicitado
    resultado = {
        desarrollador: {
            'Negative': negativo,
            'Neutral': neutro,
            'Positive': positivo
        }
    }

    return resultado

In [268]:
Sentiment_analysis("11 bit studios")

{'11 bit studios': {'Negative': 20, 'Neutral': 8, 'Positive': 18}}